In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train_sales=pd.read_csv('data/sales_train.csv.gz')
test_sales=pd.read_csv('data/test.csv.gz')

In [4]:
train_sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


# predict using the last month(october)

In [6]:
base_model=train_sales[train_sales.date_block_num==33].groupby(["shop_id", "item_id"],as_index=False).sum()[["shop_id","item_id","item_cnt_day"]]
base_model["item_cnt_day"].clip(0,20,inplace=True)
base_model

,shop_id,item_id,item_cnt_day
0,2,31,1.0
1,2,486,3.0
2,2,787,1.0
3,2,794,1.0
4,2,968,1.0
5,2,988,1.0
6,2,1075,1.0
7,2,1121,1.0
8,2,1377,1.0
9,2,1387,1.0


In [7]:
test_sales.shape

(214200, 3)

In [8]:
submission=pd.merge(test_sales, base_model, how="left", left_on=["shop_id","item_id"], right_on = ["shop_id","item_id"])
submission=submission[["ID","item_cnt_day"]]
submission.columns=["ID","item_cnt_month"]
submission.fillna(0,inplace=True)
submission.shape

(214200, 2)

In [13]:
submission.head()

,ID,item_cnt_month
0,0,0.0
1,1,0.0
2,2,1.0
3,3,0.0
4,4,0.0


In [14]:
submission.to_csv('base_model.csv',index=False)
# Public Score 1.16777
#  Private Score 1.17272

In [16]:
def get_submission_using_previous_month(month):
    base_model=train_sales[train_sales.date_block_num==month].groupby(["shop_id", "item_id"],as_index=False).sum()[["shop_id","item_id","item_cnt_day"]]
    base_model["item_cnt_day"].clip(0,20,inplace=True)
    submission=pd.merge(test_sales, base_model, how="left", left_on=["shop_id","item_id"], right_on = ["shop_id","item_id"])
    submission=submission[["ID","item_cnt_day"]]
    submission.columns=["ID","item_cnt_month"]
    submission.fillna(0,inplace=True)
    return submission

In [18]:
submission2=get_submission_using_previous_month(32)
submission2.to_csv('base_model32.csv',index=False)